In [4]:
%load_ext autoreload
%autoreload 2

from amber.adapters.text_snippet_dataset import TextSnippetDataset
from amber.core.language_model import LanguageModel
from pathlib import Path

from playground.train_sae import STORE_DIR

MODEL_ID = "sshleifer/tiny-gpt2"  # tiny model for quick experimentation
HF_DATASET = "roneneldan/TinyStories"

CACHE_DIR = Path("./store/tinystories")
STORE_DIR = Path("./store/tiny-gpt2")
DEVICE = 'cpu'
DATA_SPLIT = "train"
TEXT_FIELD = "text"
DATA_LIMIT = 200  # keep small for a quick demo
MAX_LENGTH = 128

model = LanguageModel.from_huggingface(MODEL_ID)
model.model.to(DEVICE)

dataset = TextSnippetDataset.from_huggingface(
    HF_DATASET,
    split=DATA_SPLIT,
    cache_dir=str(CACHE_DIR),
    text_field=TEXT_FIELD,
    limit=DATA_LIMIT,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Saving the dataset (1/1 shards): 100%|██████████| 200/200 [00:00<00:00, 105543.63 examples/s]


In [5]:
import os
from amber.mechanistic.autoencoder.autoencoder import Autoencoder

sae_models = STORE_DIR / "sae_models"
last_run = os.listdir(sae_models)[-1]

sae_layer = Autoencoder.load_model(sae_models / last_run / 'final.pt')

2025-09-29 23:16:39,433 [INFO] amber.mechanistic.autoencoder.autoencoder: 
Loaded model from store/tiny-gpt2/sae_models/tinystories_20250929_231617/final.pt
n_latents=2, n_inputs=2, activation=TopK_4, tied=False


{'model': OrderedDict({'pre_bias': tensor([ 0.0267, -0.0269]), 'encoder': tensor([[ 0.3574, -0.3373],
        [-0.3586,  0.3628]]), 'latent_bias': tensor([1.0191e-03, 1.4487e-05]), 'decoder': tensor([[ 0.7611, -0.6487],
        [-0.6465,  0.7629]]), 'latents_activation_frequency': tensor([51200, 51200])}), 'n_latents': 2, 'n_inputs': 2, 'activation': 'TopK_4', 'tied': False}
